# Dependencias

In [12]:
%pip install opencv-contrib-python
%pip install plotly
import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import scipy.signal as signal
from scipy.optimize import curve_fit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Definiciones

In [13]:
# Carga el video
nombre_video = 'yoyoCL.mp4' # 'yoyoCL.mp4' | 'yoyoCLEdit.mp4'
cap = cv2.VideoCapture('Videos/' + nombre_video) # Local
if not cap.isOpened():
  cap = cv2.VideoCapture('/content/' + nombre_video) # Colab

# Medidas tomadas de pixeles y metros
medida_metros = 0.10
medida_pixeles = 110 # Cantidad de pixeles en 0.1 metros
bbox_yoyo = (113, 29, 81, 88)
bbox_fijo = (89, 34, 35, 49)
const_XY = medida_metros/medida_pixeles

# Variables para guardar las posiciones y tiempos
positionsCentro = []
positionsCentro_m = []
fps = cap.get(cv2.CAP_PROP_FPS)  # Obtiene los FPS del video
frame_duration = 1.0 / fps  # Duración de cada frame en segundos # Si hay error acá, revisar video
frame_count = 0

# Trackeo

In [14]:
def calcularPosiciones(frame, bbox_yoyo, bbox_fijo, positions, positions_m):
    # Extrae las coordenadas del YoYo y el punto fijo
    (x_yoyo, y_yoyo, w_yoyo, h_yoyo) = [int(v) for v in bbox_yoyo]
    (x_fijo, y_fijo, w_fijo, h_fijo) = [int(v) for v in bbox_fijo]

    center_x_yoyo = x_yoyo + w_yoyo // 2
    center_y_yoyo = y_yoyo + h_yoyo // 2
    center_x_fijo = x_fijo + w_fijo // 2
    center_y_fijo = y_fijo + h_fijo // 2

    # Dibuja las cajas y los centros en el frame
    cv2.rectangle(frame, (x_yoyo, y_yoyo), (x_yoyo + w_yoyo, y_yoyo + h_yoyo), (0, 255, 0), 2)
    cv2.rectangle(frame, (x_fijo, y_fijo), (x_fijo + w_fijo, y_fijo + h_fijo), (255, 0, 0), 2)
    cv2.circle(frame, (center_x_yoyo, center_y_yoyo), 5, (0, 0, 255), -1)
    cv2.circle(frame, (center_x_fijo, center_y_fijo), 5, (255, 0, 0), -1)

    # Calcula la posición relativa del YoYo respecto al punto fijo
    rel_x = center_x_yoyo - center_x_fijo
    rel_y = center_y_yoyo - center_y_fijo

    # Guarda las posiciones (convertidas a metros)
    current_time = frame_count * frame_duration
    positions.append((current_time, rel_x, -rel_y))  # Posición relativa en pixeles
    positions_m.append((current_time, rel_x * const_XY, -rel_y * const_XY))  # Posición relativa en metros

In [15]:
# Inicializa los trackers
tracker_yoyo = cv2.legacy.TrackerCSRT.create()
tracker_fijo = cv2.legacy.TrackerCSRT.create()
# Lee el primer frame del video
ret, frame = cap.read()
if not ret:
    print("No se pudo leer el video.")
    cap.release()
    cv2.destroyAllWindows()
# Seleccionar BBoxes e inicializar trackers
#bbox_yoyo = cv2.selectROI("Seleccionar objeto a rastrear (YoYo)", frame, False)
print("BBox YoYo:", bbox_yoyo)
ok_yoyo = tracker_yoyo.init(frame, bbox_yoyo)
#bbox_fijo = cv2.selectROI("Seleccionar punto fijo", frame, False)print("BBox Punto Fijo:", bbox_fijo)
ok_fijo = tracker_fijo.init(frame, bbox_fijo)
while True:
    # Lee el siguiente frame
    ret, frame = cap.read()
    if not ret:
        break

    # Actualiza ambos trackers
    ok_yoyo, bbox_yoyo = tracker_yoyo.update(frame)
    ok_fijo, bbox_fijo = tracker_fijo.update(frame)

    if ok_yoyo and ok_fijo:
        calcularPosiciones(frame, bbox_yoyo, bbox_fijo, positionsCentro, positionsCentro_m)

    # Muestra el frame con el seguimiento SOLO LOCAL
    #display_frame = cv2.resize(frame, (400, 1000))
    #cv2.imshow('Object Tracking', display_frame)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Incrementa el contador de frames
    frame_count += 1

# Libera la captura y cierra las ventanas
cap.release()
cv2.destroyAllWindows()

BBox YoYo: (113, 29, 81, 88)


# Datos

In [16]:
diff_size = 4
suavizado = True
window_size = 30
polynomial_order = 3
def crear_dataframe(positions):
    # Crea el DataFrame y aplica filtros
    df = pd.DataFrame(positions, columns=['Time (sec)', 'X', 'Y'])

    df = df[df['Time (sec)'] < 2.47]

    if suavizado:
      df['X'] = signal.savgol_filter(df['X'], window_size, polynomial_order)
      df['Y'] = signal.savgol_filter(df['Y'], window_size, polynomial_order)

    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)
    df['Delta_X'] = df['X'].diff(diff_size).fillna(0)
    df['Delta_Y'] = df['Y'].diff(diff_size).fillna(0)

    # Velocidades
    df['Speed_X'] = df['Delta_X'] / df['Delta_Time']
    df.loc[0, 'Speed_X'] = df.loc[1, 'Speed_X']
    df['Speed_Y'] = df['Delta_Y'] / df['Delta_Time']
    df.loc[0, 'Speed_Y'] = df.loc[1, 'Speed_Y']
    df['Speed_X'] = df['Speed_X'].replace([np.inf, -np.inf], 0)
    df['Speed_Y'] = df['Speed_Y'].replace([np.inf, -np.inf], 0)
    df['Speed'] = np.sqrt(df['Speed_X']**2 + df['Speed_Y']**2)
    df['Speed'] = df['Speed'].replace([np.inf, -np.inf], 0)

    if suavizado:
      df['Speed_X'] = signal.savgol_filter(df['Speed_X'], window_size, polynomial_order)
      df['Speed_Y'] = signal.savgol_filter(df['Speed_Y'], window_size, polynomial_order)

    # Calcular aceleraciones
    df['Acceleration_X'] = df['Speed_X'].diff(diff_size) / df['Delta_Time']
    df['Acceleration_X'] = df['Acceleration_X'].fillna(0)
    df['Acceleration_Y'] = df['Speed_Y'].diff(diff_size) / df['Delta_Time']
    df['Acceleration_Y'] = df['Acceleration_Y'].fillna(0)
    df['Acceleration_X'] = df['Acceleration_X'].replace([np.inf, -np.inf], 0)
    df['Acceleration_Y'] = df['Acceleration_Y'].replace([np.inf, -np.inf], 0)
    df['Acceleration'] = np.sqrt(df['Acceleration_X']**2 + df['Acceleration_Y']**2)
    df['Acceleration'] = df['Acceleration'].replace([np.inf, -np.inf], 0)

    # Separar dataframes para velocidades y aceleraciones
    df_speed = df[['Time (sec)', 'Speed_X', 'Speed_Y', 'Speed']][df['Speed'] != 0].copy()
    df_acceleration = df[['Time (sec)', 'Acceleration_X', 'Acceleration_Y', 'Acceleration']][df['Acceleration'] != 0].copy()

    #df_acceleration = df_acceleration[df_acceleration['Time (sec)'].between(0.32, 2.1)]
    if suavizado:
      df_acceleration['Acceleration_X'] = signal.savgol_filter(df_acceleration['Acceleration_X'], window_size, polynomial_order)
      df_acceleration['Acceleration_Y'] = signal.savgol_filter(df_acceleration['Acceleration_Y'], window_size, polynomial_order)

    return df, df_speed, df_acceleration

In [17]:
# Creamos los DataFrames para el punto
dfCentro, dfCentro_speed, dfCentro_acceleration = crear_dataframe(positionsCentro_m)

# Guardamos los DataFrames actualizados en archivos CSV (es para chequear después si todo está en orden)
dfCentro.to_csv('DataFrameA.csv', index=False, float_format='%.6f')
print("Datos guardados en archivo CSV.")

# Comprobación opcional de conteo de filas
#print(dfCentro_acceleration.count())
#print(dfCentro_speed.count())

Datos guardados en archivo CSV.


# Graficos

In [18]:
pio.renderers.default = 'colab'  # 'browser' | 'vscode' | colab

In [19]:
# Elegir dataframe a graficar
dfG = dfCentro[dfCentro['Time (sec)'].between(0, 2.1)].copy()
dfG_speed = dfCentro_speed[dfCentro_speed['Time (sec)'].between(0, 2.1)].copy()
#dfG_acceleration = dfCentro_acceleration.copy()
dfG_acceleration = dfCentro_acceleration[dfCentro_acceleration['Time (sec)'].between(0, 2.1)].copy()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear figura y ejes
fig = make_subplots(rows=4, cols=2, subplot_titles=("Trayectoria del Objeto", "Velocidad del Objeto", "Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))

# Grafica la trayectoria
fig.update_xaxes(title_text="Posición X (m)", range=[0.35, 0.6], row=1, col=1, scaleanchor="y") # Relación 1:1 entre X e Y
fig.update_yaxes(title_text="Posición Y (m)", row=1, col=1)
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.update_yaxes(title_text="Velocidad (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=1, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X / tiempo
fig.update_yaxes(title_text="Posición X (m)", range=[0.03,0.07], row=2, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.update_yaxes(title_text="Posición Y (m)", row=2, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=2, col=2)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(title_text="Velocidad X (m/s)", range=[-0.5,0.5], row=3, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.update_yaxes(title_text="Velocidad Y (m/s)", row=3, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=3, col=2)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.update_yaxes(title_text="Aceleración X (m/s^2)", row=4, col=1)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=1)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.update_yaxes(title_text="Aceleración Y (m/s^2)", row=4, col=2)
fig.update_xaxes(title_text="Tiempo (s)", row=4, col=2)
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines', marker=dict(color='purple')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis de Movimiento del Centro del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show()

# Ajuste de curvas

In [20]:
# Ajuste de curva para aceleracion Y
def fit_acceleration_curve(df):
    def model_func(x, a):
        return np.full_like(x, a)

    x_data = df['Time (sec)'].values
    y_data = df['Acceleration_Y'].values

    popt, pcov = curve_fit(model_func, x_data, y_data)

    # y_pred = model_func(x_data, *popt)
    # ss_res = np.sum((y_data - y_pred) ** 2)
    # ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    # r_squared = 1 - (ss_res / ss_tot)

    residuals = y_data - model_func(x_data, *popt)
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    #return popt, pcov, r_squared

    # GRAFICO DE EJEMPLO
    x_fit = np.linspace(min(x_data), max(x_data), 1000)
    y_fit = model_func(x_fit, *popt)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='Datos originales',marker=dict(size=8,opacity=0.6,color='blue')))
    fig.add_trace(go.Scatter(x=x_fit,y=y_fit,mode='lines',name=f'Valor constante = {popt[0]:.4f}',line=dict(color='red', width=2)))
    fig.update_layout(title=dict(text='Ajuste constante para datos de aceleración',x=0.5,xanchor='center'),xaxis_title='Tiempo (s)',yaxis_title='Aceleración Y',showlegend=True,legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white',hovermode='x unified')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.show()

    return popt, pcov, r_squared

#popt, pcov, r_squared = fit_acceleration_curve(dfCentro_acceleration)
popt, pcov, r_squared = fit_acceleration_curve(dfCentro_acceleration[dfCentro_acceleration['Time (sec)'].between(0.32, 2)])
print(popt)
print(pcov)
print(r_squared)


dfCentro_acceleration[dfCentro_acceleration['Time (sec)'].between(0.32, 2.1)].var()

[-9.19842182]
[[0.05092158]]
-2.220446049250313e-16


Time (sec)        0.267500
Acceleration_X    0.328945
Acceleration_Y    6.430742
Acceleration      6.661648
dtype: float64